In [2]:
import numpy as np
from numpy import linalg as LA

# Usando lo visto en clase

# Sustituciones

def SustitucionDelante(Mat, b):

    n = Mat.shape[0]
    x = np.zeros(n, dtype=float)
    for i in range(n):
        s = 0.0
        for j in range(i):
            s += Mat[i, j] * x[j]
        x[i] = (b[i] - s) / Mat[i, i]
    return x


def SustitucionAtras(Mat, b):

    n = Mat.shape[0]
    x = np.zeros(n, dtype=float)
    for i in range(n-1, -1, -1):
        s = 0.0
        for j in range(i+1, n):
            s += Mat[i, j] * x[j]
        x[i] = (b[i] - s) / Mat[i, i]
    return x

# Función básica LU sin pivoteo

def LU(A):

    n = A.shape[0]
    U = A.copy().astype(float)
    L = np.eye(n)
    for i in range(n):
        if U[i, i] == 0:
            raise LA.LinAlgError(f"Pivote cero en fila {i}")
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] = U[j, i:] - L[j, i] * U[i, i:]
    return L, U


def SolverLU(A, b):

    L, U = LU(A)
    y = SustitucionDelante(L, b)
    x = SustitucionAtras(U, y)
    return x

# Pivoteo parcial

def Permutaciones_Parcial(A, b):

    n = A.shape[0]
    U = A.copy().astype(float)
    b_p = b.copy().astype(float)
    P = np.eye(n)
    for j in range(n-1):

        k = np.argmax(np.abs(U[j:, j])) + j
        if U[k, j] == 0:
            raise LA.LinAlgError("Matriz singular")

        U[[j, k], :] = U[[k, j], :]
        P[[j, k], :] = P[[k, j], :]
        b_p[[j, k]]   = b_p[[k, j]]
    return P, U, b_p


def SolverLU_Partial(A, b):

    P, U, b_p = Permutaciones_Parcial(A, b)

    L, U2 = LU(U)
    y = SustitucionDelante(L, b_p)
    x = SustitucionAtras(U2, y)
    return x

# Pivoteo total

def Permutaciones_Total(A, b):

    n = A.shape[0]
    U = A.copy().astype(float)
    b_p = b.copy().astype(float)
    P = np.eye(n)
    Q = np.eye(n)
    for k in range(n-1):
        # submatriz |U[k:, k:]|
        sub = np.abs(U[k:, k:])
        i_sub, j_sub = np.unravel_index(np.argmax(sub), sub.shape)
        i = i_sub + k
        j = j_sub + k
        if U[i, j] == 0:
            raise LA.LinAlgError("Matriz singular")
        # intercambio filas en U, P, b_p
        U[[k, i], :] = U[[i, k], :]
        P[[k, i], :] = P[[i, k], :]
        b_p[[k, i]]   = b_p[[i, k]]
        # intercambio columnas en U y Q
        U[:, [k, j]] = U[:, [j, k]]
        Q[:, [k, j]] = Q[:, [j, k]]
    return P, Q, U, b_p


def SolverLU_Total(A, b):

    P, Q, U, b_p = Permutaciones_Total(A, b)

    L, U2 = LU(U)
    y = SustitucionDelante(L, b_p)
    z = SustitucionAtras(U2, y)
    x = Q @ z
    return x



# Resolución de los sistemas

if __name__ == "__main__":
    sistemas = {
        'a': (np.array([[4, -1, 3], [-8, 4, -7], [12, 1, 8]], float),
              np.array([-8, 19, -19], float)),
        'b': (np.array([[1, 4, -2, 1],
                        [-2, -4, -3, 1],
                        [1, 16, -17, 9],
                        [2, 4, -9, -3]], float),
              np.array([3.5, -2.5, 15, 10.5], float)),
        'c': (np.array([[4, 5, -1, 2, 3],
                        [12, 13, 0, 10, 3],
                        [-8, -8, 5, -11, 4],
                        [16, 18, -7, 20, 4],
                        [-4, -9, 31, -31, -1]], float),
              np.array([34, 93, -33, 131, -58], float))
    }

    for key, (A, b) in sistemas.items():
        print(f"Sistema {key}:")
        x1 = SolverLU(A, b)
        print(" - Inciso (1) LU sin pivoteo:", x1)
        x2 = SolverLU_Partial(A, b)
        print(" - Inciso (2) LU con pivoteo parcial:", x2)
        x3 = SolverLU_Total(A, b)
        print(" - Inciso (3) LU con pivoteo total:", x3)
        print()

Sistema a:
 - Inciso (1) LU sin pivoteo: [-1.  1. -1.]
 - Inciso (2) LU con pivoteo parcial: [-1.  1. -1.]
 - Inciso (3) LU con pivoteo total: [-1.  1. -1.]

Sistema b:
 - Inciso (1) LU sin pivoteo: [-0.5  1.  -0.5 -1. ]
 - Inciso (2) LU con pivoteo parcial: [-0.5  1.  -0.5 -1. ]
 - Inciso (3) LU con pivoteo total: [-0.5  1.  -0.5 -1. ]

Sistema c:
 - Inciso (1) LU sin pivoteo: [1. 2. 3. 4. 5.]
 - Inciso (2) LU con pivoteo parcial: [1. 2. 3. 4. 5.]
 - Inciso (3) LU con pivoteo total: [1. 2. 3. 4. 5.]

